# INFO

This notebook contains all the required code to fetch, process and merge data in order to populate the country_statistics table.

Each section can be run independently since the required files are stored in /temp

# Word Bank

In [1]:
import numpy as np
import pandas as pd

from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

In [8]:
def download_WDIdata_to_df(url):
    print("Dowloading")
    resp = urlopen(url)
    zipfile = ZipFile(BytesIO(resp.read()))
    print("Downloaded")

    print("Loading into a pandas dataframe")
    data = pd.read_csv(zipfile.open('WDIData.csv'))
    print("Dataframe created")

    return data

In [9]:
def load_local_WDIdata(zip_path):
    print("loading data from ", zip_path)
    zipfile = ZipFile(zip_path)
    data = pd.read_csv(zipfile.open('WDIData.csv'))
    print("Dataframe created")

    return data

### Loads the data

In [15]:
DOWNLOAD_ZIP = False

In [16]:
if DOWNLOAD_ZIP:
    ZIP_URL = "http://databank.worldbank.org/data/download/WDI_csv.zip"
    # it can take some minutes
    wb_df = download_WDIdata_to_df(ZIP_URL)

else:
    ZIP_PATH = ('input/wb/WDI_csv.zip')
    wb_df = load_local_WDIdata(ZIP_PATH)

loading data from  input/wb/WDI_csv.zip
Dataframe created


In [17]:
wb_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.783289,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.428272,87.070576,88.176836,87.342739,89.130121,89.678685,90.273687,NaN,NaN,NaN
2,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,73.942103,75.244104,77.162305,75.538976,78.741152,79.665635,80.749293,NaN,NaN,NaN
3,Arab World,ARB,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,95.939242,95.962166,96.352930,95.997833,96.649916,96.834184,97.003974,NaN,NaN,NaN
4,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,22.260538,NaN,NaN,30.277130,NaN,NaN,37.165211,NaN,NaN,NaN


### Parse the data, for each row keep only the most recent value

In [18]:
# remove the last column (always empty)
wb_df.drop("Unnamed: 64", axis=1, inplace=True)

# load the indicators we are interested in
CODES_PATH = 'input/wb/interesting_indicators.csv'
interesting_indicators = pd.read_csv(CODES_PATH)
wb_data = wb_df.loc[wb_df["Indicator Code"].isin(interesting_indicators["code"])].copy()

# the last column name is the most recent year
most_recent_year = int(wb_data.columns[-1])

# creates two additional columns
wb_data.loc[:, "Most Recent Value"] = np.nan
wb_data.loc[:, "Year"] = np.nan

# for each row, find the most recent non NaN measure
for year in range(most_recent_year, 1959, -1):
    if wb_data["Most Recent Value"].isnull().values.any():
        wb_data.loc[wb_data["Most Recent Value"].isnull(), "Year"] = int(year)
        wb_data.loc[: ,"Most Recent Value"].fillna(wb_data[str(year)], inplace=True)
    else:
        break

# drop all the colums with years
cols_to_drop = [str(year) for year in range(1960, most_recent_year+1)]

wb_data_recent = wb_data.drop(cols_to_drop, axis=1)

In [19]:
wb_data_recent.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Most Recent Value,Year
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,90.273687,2017.0
13,Arab World,ARB,Adequacy of social insurance programs (% of to...,per_si_allsi.adq_pop_tot,NaN,1960.0
17,Arab World,ARB,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,85.384220,2018.0
18,Arab World,ARB,"Adjusted net enrollment rate, primary, female ...",SE.PRM.TENR.FE,84.252780,2018.0
19,Arab World,ARB,"Adjusted net enrollment rate, primary, male (%...",SE.PRM.TENR.MA,86.506010,2018.0


### Select only countries with valid ISO (drop things like Arab World and EU) and add GID

In [22]:
# load the full list of countries with a GID
gid_list = pd.read_csv("input/gid/admn_0.csv")

# consider only the rows with a valid GID
wb_data_to_output = wb_data_recent[wb_data_recent["Country Code"].isin(gid_list["countrycode"])].copy()

# add a GID column
wb_data_to_output["GID"] = wb_data_to_output["Country Code"]

In [23]:
wb_data_to_output.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Most Recent Value,Year,GID
67164,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,97.700000,2017.0,AFG
67176,Afghanistan,AFG,Adequacy of social insurance programs (% of to...,per_si_allsi.adq_pop_tot,15.695031,2007.0,AFG
67180,Afghanistan,AFG,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,26.766000,1993.0,AFG
67181,Afghanistan,AFG,"Adjusted net enrollment rate, primary, female ...",SE.PRM.TENR.FE,13.697640,1993.0,AFG
67182,Afghanistan,AFG,"Adjusted net enrollment rate, primary, male (%...",SE.PRM.TENR.MA,38.795030,1993.0,AFG


### Save to pickle

In [24]:
# save the full table 
wb_data_to_output.to_pickle("temp/wb_out_FULL.pkl")
print("saved to temp/wb_out_FULL.pkl")

saved to temp/wb_out_FULL.pkl


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Integrated Value Survey

In [25]:
import numpy as np
import pandas as pd

## Read the data

In [27]:
%%time
IVS_data = pd.read_csv("input/IVS/Integrated_data.csv", low_memory = False)

CPU times: user 4min 4s, sys: 1min 28s, total: 5min 33s
Wall time: 7min 27s


### Load up the list of variables we want to aggregate

In [35]:
IVS_variables = pd.read_csv("input/IVS/IVS_Variable_List.csv")

In [36]:
# lets keep only the ones we are interested in 
IVS_variables = IVS_variables[IVS_variables.Included == 1]

In [37]:
IVS_variables.head()

,Included,Name,Label,Last EVS,Last WVS,Categories
0,1,A001,Important in life: Family,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
1,1,A002,Important in life: Friends,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
4,1,A005,Important in life: Work,EVS 2008,WVS6 (2010-2012),1:Very important\n2:Rather important\n3:Not ve...
8,1,A009,State of health (subjective),EVS 2008,WVS6 (2010-2012),1:Very good\n2:Good\n3:Fair\n4:Poor\n5:Very po...
29,1,A029,Important child qualities: independence,EVS 2008,WVS6 (2010-2012),0:Not mentioned\n1:Important\n-5:Missing; Unkn...


## Slice the dataframe and preprocess some columns

In [38]:
IVS_data_subset = IVS_data.loc[: , IVS_data.columns.isin(IVS_variables.Name)].copy()

### Merge S002 with S002EVS

In [39]:
WVS_wave = pd.read_csv("input/IVS/WVS_wave.csv", index_col = "code").wave.to_dict()
EVS_wave = pd.read_csv("input/IVS/EVS_wave.csv", index_col = "code").wave.to_dict()

WVS_wave[-4] = ""
EVS_wave[-4] = ""

In [40]:
for key in WVS_wave:
    IVS_data_subset.loc[IVS_data_subset['S002'] == key, ['S002']] = WVS_wave[key]
    
for key in EVS_wave:
    IVS_data_subset.loc[IVS_data_subset['S002EVS'] == key, ['S002EVS']] = EVS_wave[key]
    
IVS_data_subset.S002 += IVS_data_subset.S002EVS

IVS_data_subset.drop(['S002EVS'], axis = 1, inplace = True)

### Change labels in S001

In [41]:
IVS_data_subset.loc[IVS_data_subset['S001'] == 1, ['S001']] = "EVS"
IVS_data_subset.loc[IVS_data_subset['S001'] == 2, ['S001']] = "WVS"

IVS_data_subset.rename(columns={'S001':'Survey'}, inplace=True)

### Fix an error on X048WVS 

In [42]:
# there are some strange values who do not appear in the list of region codes
IVS_data_subset[(IVS_data_subset.X048WVS > 0) & (IVS_data_subset.X048WVS < 100)].X048WVS.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [43]:
# they all belong to CHILE
IVS_data_subset[(IVS_data_subset.X048WVS > 0) & (IVS_data_subset.X048WVS < 100)].S003.unique()

array([152])

In [44]:
# CHILE has region code 152xxx, lets fix it
IVS_data_subset.loc[(IVS_data_subset.X048WVS > 0) & (IVS_data_subset.X048WVS < 100), "X048WVS"] += 152000

In [45]:
# there is a strange region in Sweden 
IVS_data_subset[ IVS_data_subset.X048WVS == 752208 ].head()

,Survey,S002,S003,S017,A001,A002,A005,A009,A029,A032,...,X047,X048,X048WVS,X049,X051,X052,X053,x048a_n1,x048b_n2,x048c_n3
103360,WVS,1994-1998,752,1.0,2,1,1,1,0,0,...,2,-4,752208,-4,-4,-4,-4,,,
103368,WVS,1994-1998,752,1.0,1,1,1,2,1,1,...,7,-4,752208,-4,-4,-4,-4,,,
103370,WVS,1994-1998,752,1.0,1,1,1,2,1,1,...,-1,-4,752208,-4,-4,-4,-4,,,
103371,WVS,1994-1998,752,1.0,1,2,2,1,0,1,...,2,-4,752208,-4,-4,-4,-4,,,
103372,WVS,1994-1998,752,1.0,2,3,2,3,1,1,...,-1,-4,752208,-4,-4,-4,-4,,,


In [47]:
# convert 752208 to 752028
IVS_data_subset.loc[ IVS_data_subset.X048WVS == 752208, "X048WVS" ] = 752028

### Merge X048 with X048WVS

In [48]:
# check whether we have a row with valid values in both X048 and X048WVS
IVS_data_subset[(IVS_data_subset['X048'] > 0) & (IVS_data_subset['X048WVS'] > 0)]

,Survey,S002,S003,S017,A001,A002,A005,A009,A029,A032,...,X047,X048,X048WVS,X049,X051,X052,X053,x048a_n1,x048b_n2,x048c_n3


In [49]:
# nope, we can merge the two columns
# convert missing data to 0 in order to be able to sum
IVS_data_subset.loc[IVS_data_subset['X048'] < 0, ['X048']] = 0
IVS_data_subset.loc[IVS_data_subset['X048WVS'] < 0, ['X048WVS']] = 0

IVS_data_subset.X048 += IVS_data_subset.X048WVS
IVS_data_subset.drop(['X048WVS'], axis = 1, inplace = True)

# Reconvert to -5 
IVS_data_subset.loc[IVS_data_subset['X048'] == 0, ['X048']] = -5

### Rename some columns

In [50]:
IVS_data_subset.rename(columns={'S002':'Wave'}, inplace=True)
IVS_data_subset.rename(columns={'S003':'Country Numeric'}, inplace=True)
IVS_data_subset.rename(columns={'X048':'Region Numeric'}, inplace=True)
IVS_data_subset.rename(columns={'S017':'Weight'}, inplace=True)

In [51]:
IVS_data_subset.head()

,Survey,Wave,Country Numeric,Weight,A001,A002,A005,A009,A029,A032,...,X045,X047,Region Numeric,X049,X051,X052,X053,x048a_n1,x048b_n2,x048c_n3
0,WVS,1981-1984,32,0.926626,-4,-4,-4,4,0,1,...,-2,-4,32001,-4,-4,-4,-4,,,
1,WVS,1981-1984,32,0.926626,-4,-4,-4,4,0,0,...,2,-4,32001,-4,-4,-4,-4,,,
2,WVS,1981-1984,32,0.926626,-4,-4,-4,3,0,0,...,2,-4,32001,-4,-4,-4,-4,,,
3,WVS,1981-1984,32,1.352874,-4,-4,-4,2,1,1,...,2,-4,32001,-4,-4,-4,-4,,,
4,WVS,1981-1984,32,0.926626,-4,-4,-4,2,1,0,...,3,-4,32001,-4,-4,-4,-4,,,


<br>
<br>
<br>
<br>
<br>

## One-hot encoding

In [52]:
not_to_encode = ["Survey", "Wave", "Country Numeric", "Region Numeric", 
                 "Weight", "x048a_n1", "x048b_n2", "x048c_n3"] 
# the variables we want to groupby and the weights (S017)

In [53]:
one_hot = pd.get_dummies(data=IVS_data_subset.loc[: , [c for c in IVS_data_subset.columns if c not in not_to_encode] ], 
                           columns=[c for c in IVS_data_subset.columns if c not in not_to_encode])

In [54]:
one_hot.head()

,A001_-5,A001_-4,A001_-2,A001_-1,A001_1,A001_2,A001_3,A001_4,A002_-5,A002_-4,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


### Multiply by the weights

In [57]:
weighted_one_hot = one_hot.mul(IVS_data_subset["Weight"], axis=0).copy()

In [58]:
weighted_one_hot.head()

,A001_-5,A001_-4,A001_-2,A001_-1,A001_1,A001_2,A001_3,A001_4,A002_-5,A002_-4,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,0.0,0.926626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.926626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.926626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.926626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.926626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.926626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.352874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.352874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.926626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.926626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<br>
<br>
<br>
<br>
<br>

## Groupby Country

### For country statistics we can correct the numbers using S007

In [60]:
weighted_one_hot["Survey"] = IVS_data_subset["Survey"]
weighted_one_hot["Wave"] = IVS_data_subset["Wave"]
weighted_one_hot["Country Numeric"] = IVS_data_subset["Country Numeric"]

In [61]:
grouped_by_country = weighted_one_hot.groupby(["Survey", "Wave", "Country Numeric"]).mean().copy()

In [62]:
grouped_by_country["Sample Size"] = weighted_one_hot.groupby(["Survey", "Wave", "Country Numeric"]).size()

In [63]:
grouped_by_country.head()

A001_-5  A001_-4  A001_-2  A001_-1  A001_1  \
Survey Wave      Country Numeric                                               
EVS    1981-1984 56                   0.0      1.0      0.0      0.0     0.0   
                 124                  0.0      1.0      0.0      0.0     0.0   
                 208                  0.0      1.0      0.0      0.0     0.0   
                 250                  0.0      1.0      0.0      0.0     0.0   
                 276                  0.0      1.0      0.0      0.0     0.0   

                                  A001_2  A001_3  A001_4  A002_-5  A002_-4  \
Survey Wave      Country Numeric                                             
EVS    1981-1984 56                  0.0     0.0     0.0      0.0      1.0   
                 124                 0.0     0.0     0.0      0.0      1.0   
                 208                 0.0     0.0     0.0      0.0      1.0   
                 250                 0.0     0.0     0.0      0.0      1.0   
                 276                 0.0     0.0     0.0      0.0      1.0   

                                  ...  X053_2  X053_3  X053_4  X053_5  X053_6  \
Survey Wave      Country Numeric  ...                                           
EVS    1981-1984 56               ...     0.0     0.0     0.0     0.0     0.0   
                 124              ...     0.0     0.0     0.0     0.0     0.0   
                 208              ...     0.0     0.0     0.0     0.0     0.0   
                 250              ...     0.0     0.0     0.0     0.0     0.0   
                 276              ...     0.0     0.0     0.0     0.0     0.0   

                                  X053_7  X053_8  X053_9  X053_10  Sample Size  
Survey Wave      Country Numeric                                                
EVS    1981-1984 56                  0.0     0.0     0.0      0.0         1145  
                 124                 0.0     0.0     0.0      0.0         1254  
                 208                 0.0     0.0     0.0      0.0         1182  
                 250                 0.0     0.0     0.0      0.0         1200  
                 276                 0.0     0.0     0.0      0.0         1305  

[5 rows x 1648 columns]

### Lets add the country names

In [64]:
country_ISO = pd.read_csv("input/IVS/ISO_3611.csv", index_col = "numeric")

In [65]:
country_ISO.head()

,Country,alpha-2,alpha-3
numeric,,,
4,Afghanistan,AF,AFG
8,Albania,AL,ALB
12,Algeria,DZ,DZA
16,American Samoa,AS,ASM
20,Andorra,AD,AND


In [66]:
names = []
ISO3 = []

for idx in grouped_by_country.index:
    names.append(country_ISO.loc[idx[2], "Country"])
    ISO3.append(country_ISO.loc[idx[2], "alpha-3"])

In [67]:
grouped_by_country["Country Name"] = names
grouped_by_country["Country ISO3"] = ISO3

### Flat the index

In [69]:
grouped_by_country = grouped_by_country.reset_index()

### Move the last columns to the front

In [71]:
columns = list(grouped_by_country.columns)

In [72]:
grouped_by_country = grouped_by_country[columns[:3] + columns[-2:] + [columns[-3]] + columns[3:-3]]

In [73]:
grouped_by_country.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,A001_-5,A001_-4,A001_-2,A001_-1,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,Belgium,BEL,1145,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EVS,1981-1984,124,Canada,CAN,1254,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EVS,1981-1984,208,Denmark,DNK,1182,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EVS,1981-1984,250,France,FRA,1200,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EVS,1981-1984,276,Germany,DEU,1305,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
# ATTENTION! Northern Ireland is separated by the rest of the UK!
grouped_by_country[grouped_by_country["Country ISO3"] == "GBR-NIR"]

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,A001_-5,A001_-4,A001_-2,A001_-1,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
15,EVS,1981-1984,909,North Ireland,GBR-NIR,312,0.0,1.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,EVS,1990-1993,909,North Ireland,GBR-NIR,304,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,EVS,1999-2001,909,North Ireland,GBR-NIR,1000,0.0,0.0,0.003159,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,EVS,2008-2010,909,North Ireland,GBR-NIR,500,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Save to pickle

In [76]:
grouped_by_country.to_pickle("temp/IVS_grouped_by_country.pkl")
print("saved to temp/IVS_grouped_by_country.pkl")

saved to temp/IVS_grouped_by_country.pkl


<br>
<br>
<br>
<br>
<br>

## Groupby Country and Region

### For regional aggregation we can not use the weighted answers

In [77]:
one_hot["Survey"] = IVS_data_subset["Survey"]
one_hot["Wave"] = IVS_data_subset["Wave"]
one_hot["Country Numeric"] = IVS_data_subset["Country Numeric"]
one_hot["Region Numeric"] = IVS_data_subset["Region Numeric"]

one_hot["NUTS1"] = IVS_data_subset["x048a_n1"]
one_hot["NUTS2"] = IVS_data_subset["x048b_n2"]
one_hot["NUTS3"] = IVS_data_subset["x048c_n3"]

In [78]:
grouped_by_country_region = one_hot.groupby(["Survey", "Wave", "Country Numeric", 
                                             "Region Numeric", "NUTS1", "NUTS2", 
                                             "NUTS3"]).mean().copy()

In [79]:
grouped_by_country_region["Sample Size"] = one_hot.groupby(["Survey", "Wave", "Country Numeric",
                                                            "Region Numeric", "NUTS1", "NUTS2", "NUTS3"]).size()

### Add the country and region names

In [81]:
region_codes = pd.read_csv("input/IVS/Region_codes.csv", index_col="Code")

In [82]:
region_codes.head()

,Region
Code,
-5,Unknown
-4,Not asked
-3,Not applicable
-2,No answer
-1,Don't know


In [85]:
region_name = []
r_names = []
r_ISO3 = []


for idx in grouped_by_country_region.index:
    region_name.append(region_codes.loc[idx[3], "Region"])
    r_names.append(country_ISO.loc[idx[2], "Country"])
    r_ISO3.append(country_ISO.loc[idx[2], "alpha-3"])
    

In [86]:
grouped_by_country_region["Country Name"] = r_names
grouped_by_country_region["Country ISO3"] = r_ISO3
grouped_by_country_region["Region Name"] = region_name

### Flat the index

In [87]:
grouped_by_country_region = grouped_by_country_region.reset_index()

### Rearrange the columns

In [88]:
columns_r = list(grouped_by_country_region.columns)

In [89]:
grouped_by_country_region = grouped_by_country_region[columns_r[:3] + columns_r[-3:-1]
                                                      + [columns_r[3]]+ [columns_r[-1]]
                                                      + columns_r[4:7]
                                                      + [columns_r[-4]]+ columns_r[7:-4]]

In [90]:
grouped_by_country_region.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,Belgium,BEL,56021,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EVS,1981-1984,56,Belgium,BEL,56022,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EVS,1981-1984,56,Belgium,BEL,56023,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EVS,1981-1984,56,Belgium,BEL,56024,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EVS,1981-1984,56,Belgium,BEL,56025,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
# ATTENTION! Northern Ireland is separated by the rest of the UK!
grouped_by_country_region[grouped_by_country_region["Country ISO3"] == "GBR-NIR"].head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
111,EVS,1981-1984,909,North Ireland,GBR-NIR,909001,NIR: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112,EVS,1981-1984,909,North Ireland,GBR-NIR,909002,NIR: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113,EVS,1981-1984,909,North Ireland,GBR-NIR,909003,NIR: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,EVS,1981-1984,909,North Ireland,GBR-NIR,909004,NIR: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,EVS,1981-1984,909,North Ireland,GBR-NIR,909005,NIR: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Save to pickle

In [133]:
grouped_by_country_region.to_pickle("temp/IVS_grouped_by_country_region.pkl")
print("saved to temp/IVS_grouped_by_country_region.pkl")

saved to temp/IVS_grouped_by_country_region.pkl


<br>
<br>
<br>
<br>
<br>

# Properties Dictionary
## We need to have the three tables in the same shape before concatenating them

In [1]:
import numpy as np
import pandas as pd
import json

# IVS Country Data

## Create dictionary with the IVS anwers descriptions

In [2]:
IVS_variables = pd.read_csv("input/IVS/IVS_Variable_List.csv")

In [3]:
IVS_variables = IVS_variables.loc[IVS_variables.Included == 1].copy()
IVS_variables.drop(["Included", "Last EVS", "Last WVS"], axis=1, inplace=True)

In [48]:
IVS_variables

,Label,Categories
Name,,
A001,Important in life: Family,"{'1': 'Very important', '2': 'Rather important..."
A002,Important in life: Friends,"{'1': 'Very important', '2': 'Rather important..."
A005,Important in life: Work,"{'1': 'Very important', '2': 'Rather important..."
A009,State of health (subjective),"{'1': 'Very good', '2': 'Good', '3': 'Fair', '..."
A029,Important child qualities: independence,"{'0': 'Not mentioned', '1': 'Important', '-5':..."
...,...,...
X049,Size of town,"{'1': '2,000 and less', '2': '2,000-5,000', '3..."
X051,Ethnic group,{}
X052,Institution of occupation,"{'1': 'Public institution', '2': 'Private busi..."


In [5]:
cat_dict_column = []

for idx, row in IVS_variables.iterrows():
    codes = {}
    for line in row.Categories.split("\n")[:-1]: # there is always a \n at the end
        try:
            key = line.split(":")[0]
            value = line.split(":")[1]
            codes[key] = value
        except:
            continue
    cat_dict_column.append(codes)

In [6]:
IVS_variables["Categories"] = cat_dict_column

In [7]:
IVS_variables.set_index("Name", inplace=True)

In [8]:
variables_dict = IVS_variables.to_dict(orient="index")

## We need to pop some keys
ZMEN_010 is not present in the dataset

the following ones are the values we aggregated on

S002
S002EVS
S001
S017

X048WVS
X048
x048a_n1
x048b_n2
x048c_n3

In [9]:
len(variables_dict.keys())

87

In [10]:
to_drop = ["ZMEN_010","S001", "S002", "S002EVS", "S003", "S017",
          "X048WVS", "X048", "x048a_n1", "x048b_n2", "x048c_n3"]

for key in to_drop:
    variables_dict.pop(key)

len(variables_dict.keys())

76

## Load Country data

In [11]:
# load the list of GID at the country level (admn_0)
country_gid = pd.read_csv("input/gid/admn_0.csv")
country_gid.head()

,country,countrycode,countrycode_alpha2,adm_level,adm_area_1,adm_area_1_code,adm_area_2,adm_area_2_code,adm_area_3,adm_area_3_code,gid
0,Aruba,ABW,AW,0,NaN,NaN,NaN,NaN,NaN,NaN,ABW
1,Anguilla,AIA,AI,0,NaN,NaN,NaN,NaN,NaN,NaN,AIA
2,Palestina,PSE,PS,0,NaN,NaN,NaN,NaN,NaN,NaN,PSE
3,Cyprus,CYP,CY,0,NaN,NaN,NaN,NaN,NaN,NaN,CYP
4,Micronesia,FSM,FM,0,NaN,NaN,NaN,NaN,NaN,NaN,FSM


In [12]:
# load the IVS data aggregated by country
IVS_country = pd.read_pickle("temp/IVS_grouped_by_country.pkl")
IVS_country.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,A001_-5,A001_-4,A001_-2,A001_-1,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,Belgium,BEL,1145,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EVS,1981-1984,124,Canada,CAN,1254,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EVS,1981-1984,208,Denmark,DNK,1182,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EVS,1981-1984,250,France,FRA,1200,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EVS,1981-1984,276,Germany,DEU,1305,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# check for strange missing countries
IVS_country[~ IVS_country["Country ISO3"].isin(country_gid.countrycode)]

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,A001_-5,A001_-4,A001_-2,A001_-1,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
15,EVS,1981-1984,909,North Ireland,GBR-NIR,312,0.0,1.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44,EVS,1990-1993,909,North Ireland,GBR-NIR,304,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,EVS,1999-2001,909,North Ireland,GBR-NIR,1000,0.0,0.0,0.003159,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,EVS,2008-2010,197,Northern Cyprus,NaN,500,0.0,0.0,0.007390,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,EVS,2008-2010,909,North Ireland,GBR-NIR,500,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
123,EVS,2008-2010,915,Kosovo,NaN,1601,0.0,0.0,0.008117,0.001159,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
160,WVS,1994-1998,101,Republika Srpska,NaN,400,0.0,0.0,0.000000,0.005000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
207,WVS,1994-1998,914,Bosnia,NaN,800,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
IVS_country_has_gid = IVS_country[IVS_country["Country ISO3"].isin(country_gid.countrycode)].copy()

In [15]:
IVS_country_has_gid['json'] = IVS_country_has_gid.apply(lambda x: x.to_dict(), axis=1)

In [49]:
IVS_country_has_gid

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,A001_-5,A001_-4,A001_-2,A001_-1,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,json
0,EVS,1981-1984,56,Belgium,BEL,1145,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"{'Survey': 'EVS', 'Wave': '1981-1984', 'Countr..."
1,EVS,1981-1984,124,Canada,CAN,1254,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"{'Survey': 'EVS', 'Wave': '1981-1984', 'Countr..."
2,EVS,1981-1984,208,Denmark,DNK,1182,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"{'Survey': 'EVS', 'Wave': '1981-1984', 'Countr..."
3,EVS,1981-1984,250,France,FRA,1200,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"{'Survey': 'EVS', 'Wave': '1981-1984', 'Countr..."
4,EVS,1981-1984,276,Germany,DEU,1305,0.0,1.0,0.000000,0.000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"{'Survey': 'EVS', 'Wave': '1981-1984', 'Countr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,WVS,2010-2014,818,Egypt,EGY,1523,0.0,0.0,0.000000,0.000,...,0.049620,0.073058,0.045724,0.052922,0.037831,0.046540,0.063399,0.017063,0.032774,"{'Survey': 'WVS', 'Wave': '2010-2014', 'Countr..."
362,WVS,2010-2014,840,United States of America (the),USA,2232,0.0,0.0,0.003548,0.000,...,0.043508,0.054001,0.043248,0.152357,0.101210,0.100090,0.122997,0.110341,0.136770,"{'Survey': 'WVS', 'Wave': '2010-2014', 'Countr..."
363,WVS,2010-2014,858,Uruguay,URY,1000,0.0,0.0,0.000000,0.001,...,0.104000,0.062000,0.057000,0.130000,0.037000,0.040000,0.065000,0.045000,0.064000,"{'Survey': 'WVS', 'Wave': '2010-2014', 'Countr..."
364,WVS,2010-2014,860,Uzbekistan,UZB,1500,0.0,0.0,0.000000,0.000,...,0.094667,0.078667,0.075333,0.078667,0.061333,0.031333,0.058000,0.042000,0.094667,"{'Survey': 'WVS', 'Wave': '2010-2014', 'Countr..."


In [43]:
IVS_country_has_gid.json

0      {'Survey': 'EVS', 'Wave': '1981-1984', 'Countr...
1      {'Survey': 'EVS', 'Wave': '1981-1984', 'Countr...
2      {'Survey': 'EVS', 'Wave': '1981-1984', 'Countr...
3      {'Survey': 'EVS', 'Wave': '1981-1984', 'Countr...
4      {'Survey': 'EVS', 'Wave': '1981-1984', 'Countr...
                             ...                        
361    {'Survey': 'WVS', 'Wave': '2010-2014', 'Countr...
362    {'Survey': 'WVS', 'Wave': '2010-2014', 'Countr...
363    {'Survey': 'WVS', 'Wave': '2010-2014', 'Countr...
364    {'Survey': 'WVS', 'Wave': '2010-2014', 'Countr...
365    {'Survey': 'WVS', 'Wave': '2010-2014', 'Countr...
Name: json, Length: 358, dtype: object

In [63]:
import copy

# now we have a flat dict, we will need to organize it
columns = IVS_country_has_gid.columns

properties = []

for idx, row in IVS_country_has_gid.iterrows():
    new_dict = {}
    
    # for each row we create a dict
    # {..., 
    #  column_id: {label: 'description',
    #              frequencies: {dict with the anwers onehot encoded}
    #.             categories: {dict with the answers codes}}, 
    #  ...}
    
    # label and categories are from variables_dict
    
    for answer_code in variables_dict.keys():
        sub_dict = {}
        # we already have label and categories
        sub_dict = variables_dict[answer_code]
        
        # we need frequencies
        # in the row json we have a flat dict
        # we want to aggregate by name of the answer
        sub_dict["Frequencies"] = {}
        
        for key, item in row.json.items():
            if key.startswith(answer_code):
                sub_dict["Frequencies"][key] = item
        new_dict[answer_code] = sub_dict
       
    properties.append(copy.deepcopy(new_dict)) 


In [65]:
df_to_save = IVS_country_has_gid[ ['Survey', 'Wave', 'Country Numeric', 
                           'Country Name', 'Country ISO3', 'Sample Size'] ].copy()

In [66]:
gid_column = []

for level_0_gid in IVS_country_has_gid['Country ISO3']:
    gid_column.append([level_0_gid])
    
df_to_save["gid"] = gid_column

In [67]:
df_to_save["Properties"] = properties

In [68]:
df_to_save.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,gid,Properties
0,EVS,1981-1984,56,Belgium,BEL,1145,[BEL],{'A001': {'Label': 'Important in life: Family'...
1,EVS,1981-1984,124,Canada,CAN,1254,[CAN],{'A001': {'Label': 'Important in life: Family'...
2,EVS,1981-1984,208,Denmark,DNK,1182,[DNK],{'A001': {'Label': 'Important in life: Family'...
3,EVS,1981-1984,250,France,FRA,1200,[FRA],{'A001': {'Label': 'Important in life: Family'...
4,EVS,1981-1984,276,Germany,DEU,1305,[DEU],{'A001': {'Label': 'Important in life: Family'...


In [70]:
for idx, row in df_to_save[df_to_save["Country ISO3"] == "ITA"].iterrows():
    print(row.Properties['A001'], "\n")

{'Label': 'Important in life: Family', 'Categories': {'1': 'Very important', '2': 'Rather important', '3': 'Not very important', '4': 'Not at all important', '-5': 'Missing; Unknown', '-4': 'Not asked in survey', '-3': 'Not applicable', '-2': 'No answer', '-1': 'Don´t know'}, 'Frequencies': {'A001_-5': 0.0, 'A001_-4': 0.9999999783424628, 'A001_-2': 0.0, 'A001_-1': 0.0, 'A001_1': 0.0, 'A001_2': 0.0, 'A001_3': 0.0, 'A001_4': 0.0}} 

{'Label': 'Important in life: Family', 'Categories': {'1': 'Very important', '2': 'Rather important', '3': 'Not very important', '4': 'Not at all important', '-5': 'Missing; Unknown', '-4': 'Not asked in survey', '-3': 'Not applicable', '-2': 'No answer', '-1': 'Don´t know'}, 'Frequencies': {'A001_-5': 0.0, 'A001_-4': 0.0, 'A001_-2': 0.0, 'A001_-1': 0.0005334439491662914, 'A001_1': 0.8812247075435845, 'A001_2': 0.10511809376071313, 'A001_3': 0.010436732079521996, 'A001_4': 0.0026869769291339154}} 

{'Label': 'Important in life: Family', 'Categories': {'1': 'V

In [71]:
df_to_save.to_pickle("to_upload/IVS_country_GID.pkl")

<br>
<br>
<br>
<br>
<br>

# IVS Region Data

### Load the IVS data grouped by region

In [72]:
IVS_region = pd.read_pickle("temp/IVS_grouped_by_country_region.pkl")
IVS_region.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_1,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10
0,EVS,1981-1984,56,Belgium,BEL,56021,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,EVS,1981-1984,56,Belgium,BEL,56022,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EVS,1981-1984,56,Belgium,BEL,56023,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,EVS,1981-1984,56,Belgium,BEL,56024,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EVS,1981-1984,56,Belgium,BEL,56025,BE: unknown,-4,-4,-4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Load the region codes to GID dictionaries and add the list of GID to the IVS data

### WVS/EVS

In [73]:
VWS_to_gid = pd.read_csv("input/gid/region_codes_to_GID_WVS_EVS.csv", low_memory=False)
VWS_to_gid.drop("Region Name", inplace=True, axis=1)
#remove empty lines
VWS_to_gid.dropna(how="all", inplace=True) 
VWS_to_gid['Country Numeric'] = VWS_to_gid['Country Numeric'].astype(np.int64)
VWS_to_gid['Region Numeric'] = VWS_to_gid['Region Numeric'].astype(np.int64)
VWS_to_gid.head()

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid
0,WVS,2010-2014,840,USA,840001,USA.20_1
1,WVS,2010-2014,840,USA,840001,USA.46_1
2,WVS,2010-2014,840,USA,840001,USA.30_1
3,WVS,2010-2014,840,USA,840001,USA.22_1
4,WVS,2010-2014,840,USA,840001,USA.40_1


In [74]:
# aggregate the gid 
VWS_to_list_gid = VWS_to_gid.groupby(list(VWS_to_gid.columns[:-1])).agg({'gid':lambda x: list(x)}).reset_index()
VWS_to_list_gid.head()

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid
0,WVS,1999-2004,50,BGD,50001,[BGD.3.1_1]
1,WVS,1999-2004,50,BGD,50002,[BGD.2.4_1]
2,WVS,1999-2004,50,BGD,50003,[BGD.1.2_1]
3,WVS,1999-2004,50,BGD,50004,[BGD.3.10_1]
4,WVS,1999-2004,50,BGD,50005,[BGD.3.17_1]


In [75]:
len(VWS_to_list_gid["Region Numeric"].unique())

372

In [76]:
# slice the IVS_region, we consider only the region for witch we have mapping to gids 
IVS_subset_VWS = pd.merge(IVS_region, VWS_to_list_gid, 
                on = ['Survey', 'Wave', 'Country Numeric', 'Country ISO3',
                                    'Region Numeric'],
                sort=False)

In [77]:
# lets check if we left something out
VWS_to_list_gid[~VWS_to_list_gid["Region Numeric"].isin(IVS_subset_VWS["Region Numeric"])]

,Survey,Wave,Country Numeric,Country ISO3,Region Numeric,gid


### NUTS

In [78]:
NUTS_to_gid = pd.read_csv("input/gid/region_codes_to_GID_NUTS.csv", low_memory=False)
NUTS_to_gid.drop(["Region Name", "Region Numeric", "NUTS3"], inplace=True, axis=1)
#remove empty lines
NUTS_to_gid.dropna(how="all", inplace=True) 
NUTS_to_gid['Country Numeric'] = NUTS_to_gid['Country Numeric'].astype(np.int64)

In [79]:
len(NUTS_to_gid.NUTS2.unique())

103

In [80]:
NUTS_to_list_gid = NUTS_to_gid.groupby(list(NUTS_to_gid.columns[:-1])).agg({'gid':lambda x: list(x)}).reset_index()
NUTS_to_list_gid.head()

,Survey,Wave,Country Numeric,Country ISO3,NUTS1,NUTS2,gid
0,EVS,2008-2010,56,BEL,BE1,BE10,[BEL.1.1_1]
1,EVS,2008-2010,56,BEL,BE2,BE21,[BEL.2.1_1]
2,EVS,2008-2010,56,BEL,BE2,BE22,[BEL.2.2_1]
3,EVS,2008-2010,56,BEL,BE2,BE23,[BEL.2.3_1]
4,EVS,2008-2010,56,BEL,BE2,BE24,[BEL.2.4_1]


In [81]:
# slice the IVS_region, we consider only the region for witch we have mapping to gids 
IVS_subset_NUTS = pd.merge(IVS_region, NUTS_to_list_gid, 
                on = ['Survey', 'Wave', 'Country Numeric', 'Country ISO3',
                      'NUTS1', 'NUTS2'],
                sort=False)

In [82]:
IVS_subset_NUTS.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,gid
0,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE1,BE10,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.1.1_1]
1,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE21,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.2.1_1]
2,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE22,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.2.2_1]
3,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE23,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.2.3_1]
4,EVS,2008-2010,56,Belgium,BEL,-5,Unknown,BE2,BE24,-5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BEL.2.4_1]


## Concatenate the two dataframes

In [83]:
IVS_region = pd.concat([IVS_subset_VWS, IVS_subset_NUTS], ignore_index=True)
IVS_region.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Region Numeric,Region Name,NUTS1,NUTS2,NUTS3,...,X053_2,X053_3,X053_4,X053_5,X053_6,X053_7,X053_8,X053_9,X053_10,gid
0,WVS,1999-2004,50,Bangladesh,BGD,50001,BD: Dhaka,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.1_1]
1,WVS,1999-2004,50,Bangladesh,BGD,50002,BD: Chittagong,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.2.4_1]
2,WVS,1999-2004,50,Bangladesh,BGD,50003,BD: Barisal,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.1.2_1]
3,WVS,1999-2004,50,Bangladesh,BGD,50004,BD: Mymensingh,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.10_1]
4,WVS,1999-2004,50,Bangladesh,BGD,50005,BD: Tangail,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[BGD.3.17_1]


<br>
<br>
<br>

## Create dictionary with the anwers descriptions

In [84]:
IVS_variables = pd.read_csv("input/IVS/IVS_Variable_List.csv")

In [85]:
IVS_variables = IVS_variables.loc[IVS_variables.Included == 1].copy()
IVS_variables.drop(["Included", "Last EVS", "Last WVS"], axis=1, inplace=True)

In [86]:
cat_dict_column = []

for idx, row in IVS_variables.iterrows():
    codes = {}
    for line in row.Categories.split("\n")[:-1]: # there is always a \n at the end
        try:
            key = line.split(":")[0]
            value = line.split(":")[1]
            codes[key] = value
        except:
            continue
    cat_dict_column.append(codes)

In [87]:
IVS_variables["Categories"] = cat_dict_column

In [88]:
IVS_variables.set_index("Name", inplace=True)

In [89]:
variables_dict = IVS_variables.to_dict(orient="index")

## We need to pop some keys
ZMEN_010 is not present in the dataset

the following ones are the values we aggregated on

S002
S002EVS
S001
S017

X048WVS
X048
x048a_n1
x048b_n2
x048c_n3

In [90]:
len(variables_dict.keys())

87

In [91]:
to_drop = ["ZMEN_010", "S002", "S002EVS", "S001", "S017",
          "X048WVS", "X048", "x048a_n1", "x048b_n2", "x048c_n3"]


for key in to_drop:
    variables_dict.pop(key)

len(variables_dict.keys())

77

### Dump the answers in a json

In [92]:
IVS_region['json'] = IVS_region.apply(lambda x: x.to_dict(), axis=1)

In [93]:
import copy

# now we have a flat dict, we will need to organize it
properties = []

for idx, row in IVS_region.iterrows():
    new_dict = {}
    
    # for each row we create a dict
    # {..., 
    #  column_id: {label: 'description',
    #              frequencies: {dict with the anwers onehot encoded}
    #.             categories: {dict with the answers codes}}, 
    #  ...}
    
    # label and categories are from variables_dict
    
    # add the original region labels
    new_dict["original_region_code"]= {
        'Region Numeric': row.json['Region Numeric'],
        'Region Name': row.json['Region Name'],
        'NUTS1': row.json['NUTS1'],
        'NUTS2': row.json['NUTS2'],
        'NUTS3': row.json['NUTS3'],
    }
    
    
    for answer_code in variables_dict.keys():
        
        # we already have label and categories
        sub_dict = variables_dict[answer_code]
        
        # we need frequencies
        # in the row json we have a flat dict
        # we want to aggregate by name of the answer
        sub_dict["Frequencies"] = {}
        
        for key, item in row.json.items():
            if key.startswith(answer_code):
                sub_dict["Frequencies"][key] = item
        new_dict[answer_code] = sub_dict
        
    properties.append(copy.deepcopy(new_dict))    

In [96]:
df_to_save = IVS_region[['Survey', 'Wave', 'Country Numeric', 
                           'Country Name', 'Country ISO3', 'Sample Size', 'gid']].copy()
df_to_save["Properties"] = properties

In [97]:
df_to_save.head()

,Survey,Wave,Country Numeric,Country Name,Country ISO3,Sample Size,gid,Properties
0,WVS,1999-2004,50,Bangladesh,BGD,199,[BGD.3.1_1],{'original_region_code': {'Region Numeric': 50...
1,WVS,1999-2004,50,Bangladesh,BGD,121,[BGD.2.4_1],{'original_region_code': {'Region Numeric': 50...
2,WVS,1999-2004,50,Bangladesh,BGD,105,[BGD.1.2_1],{'original_region_code': {'Region Numeric': 50...
3,WVS,1999-2004,50,Bangladesh,BGD,128,[BGD.3.10_1],{'original_region_code': {'Region Numeric': 50...
4,WVS,1999-2004,50,Bangladesh,BGD,81,[BGD.3.17_1],{'original_region_code': {'Region Numeric': 50...


In [98]:
df_to_save.to_pickle("to_upload/IVS_region_GID.pkl")

<br>
<br>
<br>
<br>
<br>

# WB Data

## We need to aggregate the data

In [112]:
wb_data = pd.read_pickle("temp/wb_out_FULL.pkl")
wb_data

,Country Name,Country Code,Indicator Name,Indicator Code,Most Recent Value,Year,GID
67164,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,9.770000e+01,2017.0,AFG
67176,Afghanistan,AFG,Adequacy of social insurance programs (% of to...,per_si_allsi.adq_pop_tot,1.569503e+01,2007.0,AFG
67180,Afghanistan,AFG,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,2.676600e+01,1993.0,AFG
67181,Afghanistan,AFG,"Adjusted net enrollment rate, primary, female ...",SE.PRM.TENR.FE,1.369764e+01,1993.0,AFG
67182,Afghanistan,AFG,"Adjusted net enrollment rate, primary, male (%...",SE.PRM.TENR.MA,3.879503e+01,1993.0,AFG
...,...,...,...,...,...,...,...
377097,Zimbabwe,ZWE,Surface area (sq. km),AG.SRF.TOTL.K2,3.907600e+05,2018.0,ZWE
377226,Zimbabwe,ZWE,Urban population,SP.URB.TOTL,4.650663e+06,2018.0,ZWE
377227,Zimbabwe,ZWE,Urban population (% of total population),SP.URB.TOTL.IN.ZS,3.220900e+01,2018.0,ZWE
377228,Zimbabwe,ZWE,Urban population growth (annual %),SP.URB.GROW,1.323892e+00,2018.0,ZWE


In [113]:
wb_data["Most Recent Value"] = wb_data["Most Recent Value"].fillna("")

In [115]:
import copy

properties = []
for idx, row in wb_data.iterrows():
    properties.append(copy.deepcopy({row["Indicator Code"]: {"Indicator Name": row["Indicator Name"],
                                               "Most Recent Value": row["Most Recent Value"],
                                               "Year": row["Year"]
    }}))

wb_data["properties"] = properties

In [116]:
wb_data.drop(["Indicator Code", "Indicator Name", "Most Recent Value", "Year"], axis=1, inplace=True)

In [117]:
def merge_list_of_dict(l):
    d = {}
    for elem in l:
        d.update(elem)
    return d

In [118]:
wb_data_aggr = wb_data.groupby(["Country Name", "Country Code", "GID"]).agg({
    'properties':lambda x: merge_list_of_dict(list(x))}).reset_index()

In [119]:
# convert the GID to list
wb_data_aggr.GID = wb_data_aggr.GID.apply(lambda x: [x])
# rename some columns
wb_data_aggr = wb_data_aggr.rename(columns={"GID": "gid",
                                            "properties": "Properties",
                                            "Country Code":"Country ISO3"})

In [120]:
# add source, year and samplesize column
wb_data_aggr["Source"] = "WB"
wb_data_aggr["Year"] = 2020
wb_data_aggr["Sample Size"] = -1
wb_data_aggr["adm level"] = 0
wb_data_aggr.head()

,Country Name,Country ISO3,gid,Properties,Source,Year,Sample Size,adm level
0,Afghanistan,AFG,[AFG],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
1,Albania,ALB,[ALB],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
2,Algeria,DZA,[DZA],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
3,American Samoa,ASM,[ASM],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
4,Andorra,AND,[AND],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0


In [130]:
# save to pickle
wb_data_aggr.to_pickle("to_upload/WB_data_aggr.pkl")

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Merge the data

In [124]:
import pandas as pd

### IVS country

In [125]:
IVS_country = pd.read_pickle("to_upload/IVS_country_GID.pkl")
IVS_country = IVS_country.rename(columns={"Survey": "Source",
                                          "Wave": "Year"})
IVS_country.drop("Country Numeric", axis=1, inplace=True)
IVS_country["adm level"] = 0
IVS_country.head()

,Source,Year,Country Name,Country ISO3,Sample Size,gid,Properties,adm level
0,EVS,1981-1984,Belgium,BEL,1145,[BEL],{'A001': {'Label': 'Important in life: Family'...,0
1,EVS,1981-1984,Canada,CAN,1254,[CAN],{'A001': {'Label': 'Important in life: Family'...,0
2,EVS,1981-1984,Denmark,DNK,1182,[DNK],{'A001': {'Label': 'Important in life: Family'...,0
3,EVS,1981-1984,France,FRA,1200,[FRA],{'A001': {'Label': 'Important in life: Family'...,0
4,EVS,1981-1984,Germany,DEU,1305,[DEU],{'A001': {'Label': 'Important in life: Family'...,0


### IVS region

In [126]:
IVS_region = pd.read_pickle("to_upload/IVS_region_GID.pkl")
IVS_region = IVS_region.rename(columns={"Survey": "Source",
                                          "Wave": "Year"})
IVS_region.drop("Country Numeric", axis=1, inplace=True)
IVS_region["adm level"] = 1
IVS_region.head()

,Source,Year,Country Name,Country ISO3,Sample Size,gid,Properties,adm level
0,WVS,1999-2004,Bangladesh,BGD,199,[BGD.3.1_1],{'original_region_code': {'Region Numeric': 50...,1
1,WVS,1999-2004,Bangladesh,BGD,121,[BGD.2.4_1],{'original_region_code': {'Region Numeric': 50...,1
2,WVS,1999-2004,Bangladesh,BGD,105,[BGD.1.2_1],{'original_region_code': {'Region Numeric': 50...,1
3,WVS,1999-2004,Bangladesh,BGD,128,[BGD.3.10_1],{'original_region_code': {'Region Numeric': 50...,1
4,WVS,1999-2004,Bangladesh,BGD,81,[BGD.3.17_1],{'original_region_code': {'Region Numeric': 50...,1


### WB data

In [127]:
wb_data_aggr = pd.read_pickle("to_upload/WB_data_aggr.pkl")
wb_data_aggr.head()

,Country Name,Country ISO3,gid,Properties,Source,Year,Sample Size,adm level
0,Afghanistan,AFG,[AFG],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
1,Albania,ALB,[ALB],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
2,Algeria,DZA,[DZA],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
3,American Samoa,ASM,[ASM],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
4,Andorra,AND,[AND],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0


### Merge

In [129]:
country_statistics = pd.concat([wb_data_aggr, IVS_country, IVS_region], ignore_index=True)
country_statistics.to_pickle("to_upload/country_statistics.pkl", protocol = 3)
country_statistics

,Country Name,Country ISO3,gid,Properties,Source,Year,Sample Size,adm level
0,Afghanistan,AFG,[AFG],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
1,Albania,ALB,[ALB],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
2,Algeria,DZA,[DZA],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
3,American Samoa,ASM,[ASM],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
4,Andorra,AND,[AND],{'EG.ELC.ACCS.ZS': {'Indicator Name': 'Access ...,WB,2020,-1,0
...,...,...,...,...,...,...,...,...
1043,United Kingdom of Great Britain and Northern I...,GBR,"[GBR.4.13_1, GBR.4.15_1, GBR.4.5_1, GBR.4.21_1...",{'original_region_code': {'Region Numeric': -5...,EVS,2008-2010,22,1
1044,United Kingdom of Great Britain and Northern I...,GBR,"[GBR.3.3_1, GBR.3.5_1, GBR.3.7_1, GBR.3.10_1, ...",{'original_region_code': {'Region Numeric': -5...,EVS,2008-2010,43,1
1045,United Kingdom of Great Britain and Northern I...,GBR,"[GBR.3.4_1, GBR.3.6_1, GBR.3.8_1, GBR.3.9_1, G...",{'original_region_code': {'Region Numeric': -5...,EVS,2008-2010,43,1
1046,United Kingdom of Great Britain and Northern I...,GBR,"[GBR.3.4_1, GBR.3.13_1, GBR.3.17_1, GBR.3.20_1...",{'original_region_code': {'Region Numeric': -5...,EVS,2008-2010,6,1


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# UPLOAD TO DB

In [1]:
import pandas as pd
import json
import psycopg2

In [187]:
def create_country_stats_table(cur):
    sql_create_country_stats_table = """ 
        CREATE TABLE IF NOT EXISTS country_statistics (
            source text NOT NULL,
            year text NOT NULL,
            country text NOT NULL,
            countrycode text NOT NULL, 
            adm_level text NOT NULL,
            gid text[] NOT NULL,
            samplesize int,
            properties jsonb NOT NULL,
            UNIQUE (gid, source, year, adm_level),
            PRIMARY KEY (gid, source, year, adm_level)
        )"""

    cur.execute(sql_create_country_stats_table)

    print("country_statistics table created")

In [188]:
def drop_country_stats_table(cur):
    sql_drop_country_stats_table = """DROP TABLE country_statistics"""
    
    try:
        cur.execute(sql_drop_country_stats_table)
        print("country_statistics table deleted")
    except:
        print("country_statistics not present")

In [189]:
def import_data(cur,data):
    
    INSERT_STATEMENT = 'INSERT INTO country_statistics \
                            (source, year, \
                             country, countrycode,  \
                             adm_level, \
                             gid, samplesize, \
                             properties) \
                        VALUES (%s, %s, \
                                %s, %s, \
                                %s, \
                                %s, %s, \
                                %s);'
    

    for idx, row in data.iterrows():
        cur.execute(INSERT_STATEMENT, (row['Source'], row['Year'],
                                       row['Country Name'], row['Country ISO3'], 
                                       row['adm level'],
                                       row['gid'], row['Sample Size'], 
                                       json.dumps(row["Properties"])))

In [190]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

In [2]:
# loads data
country_stats = pd.read_pickle("to_upload/country_statistics.pkl")

In [192]:
drop_country_stats_table(cur)
conn.commit()

create_country_stats_table(cur)
conn.commit()

import_data(cur, country_statistics)
conn.commit()
print("data uploaded")

country_statistics table deleted
country_statistics table created
data uploaded


## Test DB

In [3]:
# Connect to covid19db.org
conn = psycopg2.connect(
    host='covid19db.org',
    port=5432,
    dbname='covid19db_adm_play',
    user='covid19db_adm_rw',
    password='fGt962FdeG2yXj3c4d3'
)

cur = conn.cursor()

In [4]:
cur.execute("""SELECT * FROM country_statistics""")

data = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
data.head()

,source,year,country,countrycode,adm_level,gid,samplesize,properties
0,WB,2020,Afghanistan,AFG,0,[AFG],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
1,WB,2020,Albania,ALB,0,[ALB],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
2,WB,2020,Algeria,DZA,0,[DZA],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
3,WB,2020,American Samoa,ASM,0,[ASM],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
4,WB,2020,Andorra,AND,0,[AND],-1,"{'EN.POP.DNST': {'Year': 2018.0, 'Indicator Na..."
